In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as st
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
#import pyarrow.parquet as pq
import glob,pickle
import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['test', 'train']


In [2]:
allFiles1 = glob.glob("./input/train/*.csv")
train = pd.DataFrame()
i=1
for file_ in allFiles1:
        df = pd.read_csv(file_)
        #df = pq.read_table(file_).to_pandas()
        train=pd.concat([train,df])
        if i%10==0:
            print(i)
        i+=1
        
del df

10
20
30
40


In [3]:
train.shape

(1458643, 156)

In [4]:
Train, Test = train_test_split(train, test_size = 0.2)
del train
X_train = Train.drop(['log_trip_duration'], axis=1)
Y_train = Train["log_trip_duration"]
del Train

X_test = Test.drop(['log_trip_duration'], axis=1)
Y_test = Test["log_trip_duration"]
del Test

In [5]:
dtrain = xgb.DMatrix(X_train, label=Y_train)
del X_train 
del Y_train
dvalid = xgb.DMatrix(X_test, label=Y_test)
del X_test 
del Y_test
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
del dvalid

In [27]:
xgb_pars = {'min_child_weight': 5, 'eta': 0.5, 'colsample_bytree': 0.9, 
            'max_depth': 15,
'subsample': 0.9, 'lambda': 100000, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
'eval_metric': 'rmse', 'objective': 'reg:linear','seed':1234}

#0.37967
#xgb_pars = {'min_child_weight': 5, 'eta': 0.5, 'colsample_bytree': 0.9, 
#            'max_depth': 15,
#'subsample': 0.9, 'lambda': 100000, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
#'eval_metric': 'rmse', 'objective': 'reg:linear','seed':1234}


model = xgb.train(xgb_pars, dtrain, 10000000, watchlist, early_stopping_rounds=100,
      maximize=False, verbose_eval=1)
print('Modeling RMSLE %.5f' % model.best_score)

[0]	train-rmse:3.33914	valid-rmse:3.33739
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[1]	train-rmse:1.93074	valid-rmse:1.92907
[2]	train-rmse:1.20426	valid-rmse:1.20343
[3]	train-rmse:0.830804	valid-rmse:0.830614
[4]	train-rmse:0.648222	valid-rmse:0.648479
[5]	train-rmse:0.554184	valid-rmse:0.554768
[6]	train-rmse:0.507228	valid-rmse:0.50813
[7]	train-rmse:0.480145	valid-rmse:0.48132
[8]	train-rmse:0.462886	valid-rmse:0.464291
[9]	train-rmse:0.451601	valid-rmse:0.453176
[10]	train-rmse:0.443366	valid-rmse:0.44512
[11]	train-rmse:0.437345	valid-rmse:0.439189
[12]	train-rmse:0.432677	valid-rmse:0.434646
[13]	train-rmse:0.428927	valid-rmse:0.431004
[14]	train-rmse:0.425614	valid-rmse:0.427791
[15]	train-rmse:0.423342	valid-rmse:0.425556
[16]	train-rmse:0.420617	valid-rmse:0.422945
[17]	train-rmse:0.418297	valid-rmse:0.420682
[18]	train-rmse:0.416294	valid-rmse:0.418755
[19]	train-rmse:0.

In [28]:
import pickle
pickle.dump(model, open("nyctaxi1.dat", "wb"))

In [4]:
allFiles1 = glob.glob("./input/test/*.csv")
test = pd.DataFrame()
i=1

for file_ in allFiles1:
        df = pd.read_csv(file_)
        #df = pq.read_table(file_).to_pandas()
        test=pd.concat([test,df])
        if i%10==0:
            print(i)
        i+=1   

del df
print(test.shape)

10
20
(625134, 156)


In [5]:
model = pickle.load(open("nyctaxi.dat", "rb"))

In [6]:
test=test.reset_index(drop=True)
test_id=test['id']
test_data=test.drop(['id'], axis=1)
dtest = xgb.DMatrix(test_data)
del test_data
pred = model.predict(dtest)
#pred = model.predict(dtest,ntree_limit=model.best_ntree_limit)
pred = np.exp(pred) - 1
submission = pd.concat([test_id, pd.DataFrame(pred)],axis=1)
submission.columns = ['id','trip_duration']
#submission['trip_duration'] = submission.apply(lambda x : 1 if (x['trip_duration'] <= 0) else x['trip_duration'], axis = 1)
submission.to_csv("submission.csv", index=False)

In [29]:
test.shape

(625134, 156)